# NBA Position Modeling
**Authors:** Dan Tong   **Student Pace:** FLEX

## Overview
The goal of this project is to build models to help classify and identify NBA players as the primary position that they played. In today's NBA, positions are more fluid and the game is often described as "position-less." I'd like to examine players from around 20 seasons ago and compare how positions have changed as the game has evolved and as rules have been added. One rule change around "hand checking," which is a foul describing illegal contact from a player using their hands to impede an opponent's movement with the ball, was implemented in the 2004-2005 season. Through these model, I aim to provide recommendations to help teams build rosters based on positional needs and also attempt to showcase this trend in playstyle.

Using these features, I will build a Log Model,  Decision Tree Classifier Model and then a Random Forest Classifier.

#### Dataset was created from CSV files from Basketball-Reference.com for the NBA seasons 1999-2004

> * **TOV% (Turnover Rate):**             The percentage of a team's or player's possessions that end in a turnover
> * **USG% (Usage Rate):**                An estimate of the percentage of team plays used by a player while he was on the floor
> * **DBPM (Defensive Box Plus/Minus):**  An estimate of the per 100 possessions value of a player on defense
> * **BPM (Box Plus/Minus):**             A rate stat about a player's value per 100 possessions played on both ends of the court. League average is defined as 0.0
> * **3PA:**                              Three Point shots attempted per game
> * **2P%:**                              Two Point Field Goal Percentage
> * **FT%:**                              Free Throw Percentage
> * **TRB:**                              Rebounds per game
> * **AST:**                              Assists per game
> * **STL:**                              Steals per game
> * **BLK:**                              Blocks per game
> * **TOV:**                              Turnovers per game
> * **PF:**                               Personal Fouls per game
> * **PTS:**                              Points per game

I cleaned and created this dataframe from Basketball-Reference data. If you'd like to take a look at how I created it, you can check the "NBApositionsEDA" notebook. If you'd like to take a look at the original season stats, the CSV  files are located in the "data" folder.

In [1]:
#importing general libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
#import my created dataframe of player stats from the 2000, 2001, 2002 NBA seasons
df = pd.read_csv('data/dataframe_2000s.csv')
df = df.drop('Player', axis=1)
df.head()

,TOV%,USG%,DBPM,BPM,Pos,3PA,2P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
0,12.7,22.5,-0.1,-1.2,SG,0.4,0.435,0.756,4.8,1.6,1.0,0.5,1.7,2.4,11.4
1,14.1,25.0,-0.4,2.2,SF,1.2,0.478,0.809,10.1,3.3,1.1,1.1,3.0,3.0,20.3
2,10.3,25.6,-1.1,3.6,SG,5.0,0.468,0.887,4.4,3.8,1.3,0.2,2.2,2.3,22.1
3,14.4,24.1,-0.8,-2.5,C,0.1,0.439,0.766,3.3,1.2,0.4,0.5,1.7,2.0,10.5
4,14.4,23.4,-1.2,-0.1,SG,2.8,0.472,0.877,4.0,3.4,1.4,0.2,2.6,2.3,16.9


In [4]:
print(df.info())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1227 entries, 0 to 1226
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TOV%    1227 non-null   float64
 1   USG%    1227 non-null   float64
 2   DBPM    1227 non-null   float64
 3   BPM     1227 non-null   float64
 4   Pos     1227 non-null   object 
 5   3PA     1227 non-null   float64
 6   2P%     1227 non-null   float64
 7   FT%     1227 non-null   float64
 8   TRB     1227 non-null   float64
 9   AST     1227 non-null   float64
 10  STL     1227 non-null   float64
 11  BLK     1227 non-null   float64
 12  TOV     1227 non-null   float64
 13  PF      1227 non-null   float64
 14  PTS     1227 non-null   float64
dtypes: float64(14), object(1)
memory usage: 143.9+ KB
None


,TOV%,USG%,DBPM,BPM,3PA,2P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
count,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000
mean,13.635371,19.587857,0.040587,-0.006275,1.689487,0.465604,0.748712,4.882478,2.596740,0.912551,0.600326,1.642624,2.405868,11.350204
std,3.461231,4.932719,1.101963,2.579651,1.710879,0.040932,0.098321,2.411679,1.981128,0.443465,0.609616,0.725048,0.627092,5.598998
min,6.300000,6.500000,-2.800000,-6.900000,0.000000,0.348000,0.262000,1.000000,0.200000,0.200000,0.000000,0.400000,0.800000,2.400000
25%,11.200000,16.100000,-0.800000,-1.700000,0.100000,0.438000,0.698000,3.100000,1.100000,0.600000,0.200000,1.100000,2.000000,7.100000
50%,13.200000,19.300000,0.000000,-0.100000,1.200000,0.464000,0.765000,4.300000,2.000000,0.800000,0.400000,1.500000,2.400000,9.900000
75%,15.700000,22.600000,0.800000,1.500000,3.000000,0.493000,0.817000,6.200000,3.500000,1.100000,0.800000,2.100000,2.800000,14.200000
max,29.900000,37.800000,3.600000,9.300000,8.000000,0.615000,0.950000,14.100000,10.900000,2.800000,3.700000,4.100000,4.500000,31.400000


### Baseline Model - Decision Tree

In [5]:
#importing train_test_split
from sklearn.model_selection import train_test_split

#splitting data into training and testing sets
y = df['Pos']
X = df.drop(columns=['Pos'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [15]:
#fit baseline Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(criterion='entropy')

#fit Decision Tree Classifier
dtc = dtc.fit(X_train, y_train)

#predict data
y_pred = dtc.predict(X_test)

In [16]:
#import accuracy
from sklearn.metrics import accuracy_score

print("Test Accuracy: ", accuracy_score(y_test, y_pred))

Test Accuracy:  0.8143322475570033


In [59]:
#import
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_baseline = LogisticRegression(random_state=42)

log_loss = -cross_val_score(log_baseline, X_train, y_train, scoring='neg_log_loss').mean()
log_loss

0.7373454212796389

In [14]:
#importing scaler to scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [56]:
#imports
from sklearn.neighbors import KNeighborsClassifier

#creating the model
knn_baseline = KNeighborsClassifier()

#performing cross-validation
knn_baseline_log_loss = -cross_val_score(knn_baseline, X_train_scaled, y_train, scoring='neg_log_loss').mean()
knn_baseline_log_loss

2.7367702002501035

In [41]:
#second model
knn_second_model = KNeighborsClassifier(n_neighbors=100)

knn_second_log_loss = -cross_val_score(knn_second_model, X_train_scaled, y_train, scoring='neg_log_loss').mean()
print('second model: ', knn_second_log_loss)

#third model
knn_third_model = KNeighborsClassifier(n_neighbors=25)

knn_third_log_loss = -cross_val_score(knn_third_model, X_train_scaled, y_train, scoring='neg_log_loss').mean()
print('third model: ', knn_third_log_loss)

#fourth model
knn_fourth_model = KNeighborsClassifier(n_neighbors=50)

knn_fourth_log_loss = -cross_val_score(knn_fourth_model, X_train_scaled, y_train, scoring='neg_log_loss').mean()
print('third model: ', knn_fourth_log_loss)

second model:  1.0502499072255573
third model:  1.0009815805075406
third model:  0.9705867738307254


In [43]:
#import Decision Tree
from sklearn.tree import DecisionTreeClassifier

dtc_baseline = DecisionTreeClassifier(random_state=42)

dtc_baseline_log_loss = -cross_val_score(dtc_baseline, X_train, y_train, scoring='neg_log_loss').mean()
dtc_baseline_log_loss

6.795128834216129